In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('../01_Base_Dados/AmesHousing.csv')

In [3]:
# 1. Tratamento de Outliers
# Remover as duas observações com área habitável muito alta
df = df[df['Gr Liv Area'] < 4000]

In [4]:
# 2. Feature Engineering
df['Age_at_Sale'] = df['Yr Sold'] - df['Year Built']
df['Total_SF'] = df['Total Bsmt SF'] + df['1st Flr SF'] + df['2nd Flr SF']

In [5]:
# 3. Tratamento de Nulos
# Nas colunas categóricas, NaN significa 'Nenhum'
cols_none = ['Pool QC', 'Misc Feature', 'Alley', 'Fence', 'Fireplace Qu', 'Garage Type', 
             'Garage Finish', 'Garage Qual', 'Garage Cond', 'Bsmt Qual', 'Bsmt Cond', 
             'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin Type 2', 'Mas Vnr Type']
for col in cols_none:
    df[col] = df[col].fillna('None')

# Nas colunas numéricas, preencher com a mediana/média
df['Lot Frontage'] = df['Lot Frontage'].fillna(df['Lot Frontage'].median())
df['Mas Vnr Area'] = df['Mas Vnr Area'].fillna(0)
df['Garage Yr Blt'] = df['Garage Yr Blt'].fillna(df['Year Built']) # Se não tem garagem, usa ano da casa

In [6]:
# 4. Transformação da Variável Alvo (Normalização Logarítmica)
y = np.log1p(df['SalePrice'])
X = df.drop(['SalePrice', 'Order', 'PID'], axis=1)

In [7]:
# 5. Codificação de Variáveis
X = pd.get_dummies(X, drop_first=True)

In [8]:
# 6. Separação Treino/Teste e Escalonamento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- SALVANDO O SCALER E OS DADOS ---
# 7. Salva o scaler
joblib.dump(scaler, '../03_Modelos_Salvos/scaler_padronizador.pkl')
# 8. Salva os dados processados em um dicionário para o Notebook 03
dados_finais = {
    'X_train': X_train_scaled,
    'X_test': X_test_scaled,
    'y_train': y_train,
    'y_test': y_test,
    'features': X.columns.tolist()
}
joblib.dump(dados_finais, '../01_Base_Dados/dados_finalizados.pkl')

print(f"Pré-processamento concluído! {X_train_scaled.shape[1]} features geradas.")

Pré-processamento concluído! 275 features geradas.
